In [1]:
import numpy as np
import pandas as pd

# Preprocessing
from sklearn.model_selection import train_test_split

# Model
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv("naver.csv")
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [3]:
train_nan = list(df.isna().sum())
num_nan = list(filter(lambda x:x >= 1, train_nan))

index_nan = list(filter(lambda e:train_nan[e] >= 1, range(len(train_nan))))

nan_df = pd.DataFrame({'index_in_train' : index_nan, 'column_name' : df.iloc[:, index_nan].columns, 'NAN_number' : num_nan})
display(nan_df)

,index_in_train,column_name,NAN_number
0,4,브랜드,190
1,6,세부제품특징,732
2,7,사용부위,1776
3,8,사용시간,2987
4,9,제품형태,2660
5,10,용량,2511
6,11,용기형태,3903
7,12,피부타입,1718
8,13,주요제품특징,36
9,14,모델명,1636


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   상품명           5000 non-null   object
 1   가격            5000 non-null   object
 2   상품번호          5000 non-null   int64 
 3   상품상태          5000 non-null   object
 4   브랜드           4810 non-null   object
 5   원산지           5000 non-null   object
 6   세부제품특징        4268 non-null   object
 7   사용부위          3224 non-null   object
 8   사용시간          2013 non-null   object
 9   제품형태          2340 non-null   object
 10  용량            2489 non-null   object
 11  용기형태          1097 non-null   object
 12  피부타입          3282 non-null   object
 13  주요제품특징        4964 non-null   object
 14  모델명           3364 non-null   object
 15  제조사           4275 non-null   object
 16  혜택조건          629 non-null    object
 17  세트수량          146 non-null    object
 18  반품안심케어 가입 안내  79 non-null     object
 19  이벤트   

In [5]:
# Column Select
train_columns = ["상품명", "가격", "세부제품특징", 
                 "피부타입", "사용부위", "사용시간", "제품형태"]
df = df.loc[:, train_columns]

### NaN Preprocessing

In [6]:
# 유니크값 확인 함수
def unique(column):
    result = []
    result2 = []
    tmp = column.dropna()
    
    for i in tmp:
        result += i.split(",")
        
    for j in result:
        result2.append(j.replace(" ", ""))
        
    result2 = set(result2)
    result2 = list(result2)
    result2.sort()
    
    return result2

In [7]:
# 가변수화 함수
def dummy(column):
    tmp = pd.DataFrame()
    for i in unique(column):
        x = []
        for n in column.dropna():
            if i in n:
                x.append(1)
            else:
                x.append(0)
        tmp = pd.concat([tmp, pd.DataFrame({ i : x })], axis = 1)
    return tmp

In [8]:
# 결측치 처리
df.isna().sum()

상품명          0
가격           0
세부제품특징     732
피부타입      1718
사용부위      1776
사용시간      2987
제품형태      2660
dtype: int64

In [9]:
# 세부제품특징

# parsing
unique(df['세부제품특징'])

['각질케어',
 '노폐물제거',
 '레포츠용',
 '메이크업베이스겸용',
 '모공케어',
 '미세먼지세정',
 '밀착력',
 '부드러운발림',
 '브라이트닝',
 '블랙헤드케어',
 '상쾌함',
 '안티에이징',
 '알콜프리',
 '약산성',
 '어린이겸용',
 '영양공급',
 '오일프리',
 '워터프루프세정',
 '유수분밸런스조절',
 '윤기부여',
 '자외선차단',
 '저자극',
 '주름케어',
 '지속력',
 '집중케어',
 '초미세먼지세정',
 '촉촉함(수분공급)',
 '쿨링감',
 '트러블케어',
 '편리한사용성',
 '피부강화',
 '피부결정돈',
 '피부보호',
 '피부유연',
 '피부진정',
 '피부탄력',
 '피부톤보정',
 '피부투명',
 '피지조절',
 '피지케어',
 '향',
 '화이트닝',
 '화이트헤드케어',
 '흡수력']

In [10]:
# 피부타입 : 모든피부용
df['피부타입'] = df['피부타입'].fillna('모든피부용')

In [11]:
# 사용부위

# parsing
unique(df['사용부위'])

['T존',
 'U존',
 '겨드랑이',
 '넥',
 '립용',
 '바디용',
 '스팟',
 '아이용',
 '코',
 '팔',
 '팔자주름',
 '페이스용',
 '핸드용']

In [12]:
# 사용시간

df['사용시간'] = df['사용시간'].fillna('낮, 밤')

In [13]:
# 제품형태

# parsing
unique(df['제품형태'])

['가루형',
 '거품형',
 '구슬형',
 '로션형',
 '리필형',
 '무스형',
 '밤형',
 '버터형',
 '브러쉬형',
 '비누',
 '스크럽형',
 '시트형',
 '압축/팩트형',
 '액상형',
 '오일형',
 '젤형',
 '캡슐/정',
 '쿠션형',
 '크림형',
 '패치형']

In [14]:
df.isnull().sum()

상품명          0
가격           0
세부제품특징     732
피부타입         0
사용부위      1776
사용시간         0
제품형태      2660
dtype: int64

In [15]:
# drop na
df = df.dropna()
df = df.reset_index(drop=True)
df

,상품명,가격,세부제품특징,피부타입,사용부위,사용시간,제품형태
0,[스킨푸드][당근2종키트증정] 캐롯 카로틴 카밍 워터 패드 30매 135g_당근패드,"13,000","피부진정, 피부결정돈, 상쾌함",모든피부용,페이스용,"낮, 밤",스크럽형
1,[스킨푸드][당근2종키트증정] 캐롯 카로틴 카밍 워터 패드 60매_당근패드,"26,000","피부진정, 피부결정돈, 상쾌함",모든피부용,페이스용,"낮, 밤",스크럽형
2,[1+1]바버 멀티 비타민 앰플 (2mlx7병)- 바버공식수입사,"60,000","저자극, 피부강화, 각질케어, 피부탄력, 영양공급",모든피부용,페이스용,"낮, 밤",액상형
3,데저트에센스 100% 냉압착 비정제 퓨어 호호바오일 118ml,"27,700","저자극, 피부보호",모든피부용,페이스용,"낮, 밤",오일형
4,벤튼 구아바 70 스킨 토너 패드 팩 (70매) 진정 수부지 비건 피부결,"36,000","피부결정돈, 피부진정, 흡수력, 모공케어","모든피부용, 건성, 민감성",페이스용,"낮, 밤",시트형
...,...,...,...,...,...,...,...
1802,로우퀘스트 에키네시아 클렌징 폼 150ml / 폼클렌징 저자극 각질제거,"25,000","노폐물 제거, 각질케어, 모공케어, 피지케어","민감성, 모든피부용, 지복합",페이스용,"낮, 밤",젤형
1803,동국제약 센텔리안24 더 마데카크림 45ml (시즌6) / 병풀추출물 보습 탄력 안...,"33,000","피부탄력, 영양공급, 피부진정, 안티에이징",모든피부용,페이스용,"낮, 밤",크림형
1804,[💘1+1💘알림받기 즉시할인]스와니코코 EGF/FGF 1ppm 40ml/ 피부 치밀...,"68,000","저자극, 피부탄력, 영양공급, 안티에이징, 피부강화","모든피부용, 지복합, 복합","페이스용, 팔자주름","낮, 밤",액상형
1805,믹순 병풀 스틱밤 11.5ml,"35,000","피부탄력, 피부진정",모든피부용,페이스용,"낮, 밤",밤형


### Column Preprocess

In [ ]:
# 가격
df['가격'] = df['가격'].str.replace(',', '')
df['가격'] = df['가격'].astype(int)

# 세부제품특징
tmp = dummy(df['세부제품특징'])
df = pd.concat([df, tmp], axis = 1)
df.drop(['세부제품특징'], axis=1, inplace=True)

# 피부타입
tmp = dummy(df['피부타입'])
df = pd.concat([df, tmp], axis = 1)
df.drop(['피부타입'], axis=1, inplace=True)

# 사용부위
tmp = dummy(df['사용부위'])
df = pd.concat([df, tmp], axis = 1)
df.drop(['사용부위'], axis=1, inplace=True)

# 사용시간
df['사용시간'] = df['사용시간'].str.replace('낮, 밤', '0')
df['사용시간'] = df['사용시간'].str.replace('밤, 낮', '0')
df['사용시간'] = df['사용시간'].str.replace('낮', '1')
df['사용시간'] = df['사용시간'].str.replace('밤', '2')
df['사용시간'] = df['사용시간'].astype('int')

# 제품형태
tmp = dummy(df['제품형태'])
df = pd.concat([df, tmp], axis = 1)
df.drop(['제품형태'], axis=1, inplace=True)

In [63]:
df.columns

Index(['상품명', '가격', '사용시간', '각질케어', '노폐물제거', '레포츠용', '메이크업베이스겸용', '모공케어',
       '밀착력', '부드러운발림', '브라이트닝', '블랙헤드케어', '상쾌함', '안티에이징', '알콜프리', '약산성',
       '어린이겸용', '영양공급', '오일프리', '유수분밸런스조절', '윤기부여', '저자극', '주름케어', '지속력',
       '초미세먼지세정', '촉촉함(수분공급)', '쿨링감', '트러블케어', '피부강화', '피부결정돈', '피부보호', '피부유연',
       '피부진정', '피부탄력', '피부톤보정', '피부투명', '피지조절', '피지케어', '향', '화이트닝', '화이트헤드케어',
       '흡수력', '건성', '모든피부용', '민감성', '복합', '중건성', '지복합', '지성', 'T존', 'U존', '넥',
       '바디용', '스팟', '아이용', '코', '팔자주름', '페이스용', '핸드용', '가루형', '거품형', '로션형',
       '리필형', '무스형', '밤형', '버터형', '스크럽형', '시트형', '압축/팩트형', '액상형', '오일형', '젤형',
       '캡슐/정', '쿠션형', '크림형', '패치형'],
      dtype='object')

### Modeling

In [26]:
# x, y split
target = '상품명'
x = df.drop(target, axis = 1)
y = df.loc[:, target]

# train, valid split
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.3, random_state=2022)

In [27]:
dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)

DecisionTreeClassifier()

In [65]:
import joblib

joblib.dump(dt, 'recsys_dt.pkl')

['recsys_dt.pkl']

In [28]:
pred = dt.predict(x_valid)

In [29]:
pred[0:10]

array(['[공식몰][2+1] 아누아 어성초 77 클리어 패드 70매',
       '[네이처리퍼블릭] 그린더마 티트리 시카 빅토너 500ml', '라비앙 레디언스 바이오 셀룰로오스 마스크 팩 5EA',
       '메디힐 콜라겐 채움 패드 100매', '아미퓨어 머랭 버블 클렌저 500ml',
       '파이콜로지 씨위드 버블 클레이 마스크 -모공, 각질, 피부결 5g10매 워시오프 뷰라벨',
       '[클리어런스] 잇츠스킨 타이거 시카 그린 칠 다운 카밍 수딩 패드 100ea(150g)',
       '발효보습 어바웃미 쌀 막걸리 스킨 170ml',
       '유세린 아쿠아포린 수분크림 액티브 포 드라이스킨 50ml (건성용)',
       '아벤느 시칼파트 플러스 SOS 크림 100ml 1+1'], dtype=object)

In [30]:
y_valid[0:10]

344                     [공식몰][2+1] 아누아 어성초 77 클리어 패드 70매
418                      [네이처리퍼블릭] 그린더마 티트리 시카 빅토너 500ml
652                         라비앙 레디언스 바이오 셀룰로오스 마스크 팩 5EA
545                                   메디힐 콜라겐 채움 패드 100매
1084                                아미퓨어 머랭 버블 클렌저 500ml
292         셀리맥스 시카 바하  지우개 폼 클렌징 150ml / 여드름성 피부 완화 기능성
1554    [클리어런스] 잇츠스킨 타이거 시카 그린 칠 다운 카밍 수딩 패드 100ea(150g)
1063                            [유리아쥬] 제모스 토너 250ml+GIFT
1369               유세린 아쿠아포린 수분크림 액티브 포 드라이스킨 50ml (건성용)
1503                       아벤느 시칼파트 플러스 SOS 크림 100ml 1+1
Name: 상품명, dtype: object

### 유저 데이터 넣기

In [52]:
user = [0, '노폐물 제거', '모든피부용', '페이스용', '낮, 밤', '스크럽형']

In [61]:
test_df = pd.DataFrame(columns=df.columns)
test_df

,상품명,가격,사용시간,각질케어,노폐물제거,레포츠용,메이크업베이스겸용,모공케어,밀착력,부드러운발림,...,스크럽형,시트형,압축/팩트형,액상형,오일형,젤형,캡슐/정,쿠션형,크림형,패치형


In [56]:
test_df.loc[0, :] = user
test_df

,가격,세부제품특징,피부타입,사용부위,사용시간,제품형태
0,0,노폐물 제거,모든피부용,페이스용,"낮, 밤",스크럽형


In [58]:
# 가격
try:
    test_df['가격'] = test_df['가격'].str.replace(',', '')
    test_df['가격'] = test_df['가격'].astype(int)
except:
    test_df['가격'] = test_df['가격'].astype(int)

# 세부제품특징
tmp = dummy(test_df['세부제품특징'])
test_df = pd.concat([test_df, tmp], axis = 1)
test_df.drop(['세부제품특징'], axis=1, inplace=True)

# 피부타입
tmp = dummy(test_df['피부타입'])
test_df = pd.concat([test_df, tmp], axis = 1)
test_df.drop(['피부타입'], axis=1, inplace=True)

# 사용부위
tmp = dummy(test_df['사용부위'])
test_df = pd.concat([test_df, tmp], axis = 1)
test_df.drop(['사용부위'], axis=1, inplace=True)

# 사용시간
test_df['사용시간'] = test_df['사용시간'].str.replace('낮, 밤', '0')
test_df['사용시간'] = test_df['사용시간'].str.replace('밤, 낮', '0')
test_df['사용시간'] = test_df['사용시간'].str.replace('낮', '1')
test_df['사용시간'] = test_df['사용시간'].str.replace('밤', '2')
test_df['사용시간'] = test_df['사용시간'].astype('int')

# 제품형태
tmp = dummy(test_df['제품형태'])
test_df = pd.concat([test_df, tmp], axis = 1)
test_df.drop(['제품형태'], axis=1, inplace=True)

In [59]:
test_df

,가격,사용시간,노폐물제거,모든피부용,페이스용,스크럽형
0,0,0,0,1,1,1
